In [ ]:
import json

In [ ]:
model_names = ["gemma-4b", "llava-13b", "llava-7b", "llama-11b", "internvl-14b", "internvl-8b", "internvl-2b", "internvl-1b", "qwen2-2b", "qwen-7b","qwen-3b"]

In [ ]:
# now get steering res
steer_ratio_oneword_lp = dict()
steer_ratio_oneword_noise_lp = dict()
lenn_deltas = dict()
for scale_factor in [5]:
  # model_names = ["qwen-3b", "internvl-14b", "llava-7b"]#,"gemma-12b"]

  steer_ratio_oneword_lp[scale_factor] = dict()
  steer_ratio_oneword_noise_lp[scale_factor] = dict()
  lenn_deltas[scale_factor] = dict()
  for mm in model_names:
    steer_ratio_oneword_lp[scale_factor][mm] = {}  # Changed to dict to store by layer number
    steer_ratio_oneword_noise_lp[scale_factor][mm] = {}  # Changed to dict
    lenn_deltas[scale_factor][mm] = []


  def get_logprob_deltas(data, layername, modelname):
      out_delts_corr = []
      out_delts_wrong = []
      og_data = data.get("none", {})
      sel_data = data.get(layername, {})
      lenn = 0
      lenn_bs = 0


      belief_swapped = 0
      now_wrong = 0
      for elt_key in list(sel_data.keys()):

        if 'log_prob' in list(sel_data[elt_key].keys()) and 'log_prob' in list(og_data[elt_key].keys()):

          v = sel_data[elt_key]['log_prob']
          v_og = og_data[elt_key]['log_prob']

          # print(v,v_og)

          # if (v[0] < -1e3) or (v_og[0] < -1e3) or (v[1] < -1e3) or (v_og[1] < -1e3):
          #   # pass
          #   print(v,v_og)
          # else:

          out_delts_wrong.append(v[1] - v_og[1])
          out_delts_corr.append(v[0] - v_og[0])
          lenn += 1

          if (v[1] - v[0]) > 0 :
            now_wrong += 1

          original_difference = v_og[0] - v_og[1]
          new_difference = v[0] - v[1]

          if original_difference != 0 and new_difference != 0:
            if (new_difference / original_difference) < 0:
              # if division is negative, that means belief was swapepd.
              belief_swapped += 1
            lenn_bs += 1

          else:
            print(f"{model_name} - Skipping bc 0", elt_key)
        else:
          print(f"{model_name} - Skipping bc no logprob", elt_key)
      print("now wrong == belief swapped?", now_wrong, belief_swapped)
      if lenn_bs == 0:
        lenn_bs = 1
        print(f"{model_name} - lenn_bs was 0. ")
      if lenn == 0:
        lenn = 1
        print(f"{model_name} - lenn was 0. ")

      lenn_deltas[scale_factor][model_name].append((lenn_bs, lenn))
      # print(sum(out_delts_wrong)/lenn)
      return sum(out_delts_corr)/lenn, sum(out_delts_wrong)/lenn, belief_swapped/lenn_bs, now_wrong/lenn

  for model_name in model_names:
    print(model_name)
    json_file = open(f"metadata/{model_name}_{scale_factor}.json")
    data = json.load(json_file)
    
    # Store results by layer number (not list index) to maintain correspondence
    for key_ in data.keys():
      if key_ != "none" and 'noise' not in key_:
        layer_num = int(key_.split('_')[1])
        steer_ratio_oneword_lp[scale_factor][model_name][layer_num] = get_logprob_deltas(data, key_, model_name)

    for key_ in data.keys():
      if key_ != "none" and 'noise' in key_:
        layer_num = int(key_.split('_')[1])
        steer_ratio_oneword_noise_lp[scale_factor][model_name][layer_num] = get_logprob_deltas(data, key_, model_name)



In [ ]:
sll_max = dict()
sll_noise_max = dict()
scf = 5
for modelname in model_names:
    # Find layers that exist in BOTH intervention and noise datasets
    intervention_layers = set(steer_ratio_oneword_lp[scf][modelname].keys())
    noise_layers = set(steer_ratio_oneword_noise_lp[scf][modelname].keys())
    common_layers = sorted(intervention_layers & noise_layers)
    
    # Get intervention values for common layers only
    layer_to_val = {layer: steer_ratio_oneword_lp[scf][modelname][layer][2] 
                    for layer in common_layers}
    
    # Find top 3 layers by intervention value
    top_layers = sorted(layer_to_val.keys(), key=lambda l: layer_to_val[l], reverse=True)[:3]
    
    # Store top intervention values
    vals = [layer_to_val[layer] for layer in top_layers]
    sll_max[modelname] = vals
    
    # Get corresponding noise values at the SAME layer numbers
    vals_noise = [steer_ratio_oneword_noise_lp[scf][modelname][layer][2] for layer in top_layers]
    sll_noise_max[modelname] = vals_noise


## Fig 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# model_names = sorted(list(steer_ratio_reasoning.keys())) # Sort model names alphabetically
num_models = len(model_names)

# Prepare data for plotting (Reasoning)
change_ratios = [np.mean(sll_max[model_name]) for model_name in model_names]
change_ratios_std = [np.std(sll_max[model_name]) for model_name in model_names]


# Prepare data for plotting (Reasoning Noise)
change_ratios_noise = [np.mean(sll_noise_max[model_name]) for model_name in model_names]
change_ratios_noise_std = [np.std(sll_noise_max[model_name]) for model_name in model_names]

r1 = np.arange(num_models)
bar_height = 0.45
r2 = [x + bar_height for x in r1]
r3 = [x + bar_height for x in r2]

plt.figure(figsize=(8.5,5.5))

# Plot for Reasoning
plt.barh(r1, change_ratios, height=bar_height, color='skyblue', label='Correct', xerr=change_ratios_std, capsize=2, error_kw={'ecolor': 'cornflowerblue'})
plt.barh(r2, change_ratios_noise, height=bar_height, color='peachpuff',  label='Wrong', xerr=change_ratios_noise_std, capsize=2, error_kw={'ecolor': 'sandybrown'})

# axes[0].set_xlabel('Average Steer Ratio (Reasoning)', fontweight='bold')
plt.yticks([r + bar_height for r in range(num_models)], model_names) #, rotation=90, ha="center")
plt.xticks([0.0, 0.2, 0.4, 0.6, 0.8], [0,20,40,60,80])
plt.title(f'{scf}x steered - Spatial ID Steering by Model - % Belief Swapped')
# axes[0].legend()


plt.tight_layout()
plt.show()

In [ ]:
import os
import json


og_ratio_reasoning = dict()
og_ratio_oneword = dict()

def get_ratios_from_layer(data, layername):
    sel_data = data.get(layername, {})
    # print(sel_data.values())
    verdict_dict = {"correct":0, "wrong":0, "nonsense":0}
    for elt in sel_data.values():
      v = elt['verdict']
      verdict_dict[v] += 1

    ratio_corr = verdict_dict['correct'] / (verdict_dict['correct'] + verdict_dict['wrong'] + verdict_dict['nonsense'])
    ratio_wrong = verdict_dict['wrong'] / (verdict_dict['correct'] + verdict_dict['wrong'] + verdict_dict['nonsense'])
    ratio_nonsense = verdict_dict['nonsense'] / (verdict_dict['correct'] + verdict_dict['wrong'] + verdict_dict['nonsense'])

    ratio = [ratio_corr, ratio_wrong, ratio_nonsense]

    return ratio


metadata_dir = "metadata"

#first populate accuracies
for filename in os.listdir(metadata_dir):

    if filename.endswith("_reasoning.json"):
        model_name = filename.replace("_reasoning.json", "")
        json_path = os.path.join(metadata_dir, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
            og_ratio_reasoning[model_name] = get_ratios_from_layer(data, "none")
    elif "reasoning" not in filename:

        model_name = filename.replace(".json", "")
        json_path = os.path.join(metadata_dir, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
            og_ratio_oneword[model_name] = get_ratios_from_layer(data, "none")

print(og_ratio_oneword)